In [2]:
import numpy as np
from keras.layers import  Dense, Flatten, Activation, Dropout, Embedding
from keras.layers import LSTM, TimeDistributed, Permute,Reshape, Lambda, RepeatVector, merge, Input,Multiply
from keras.utils import np_utils
from keras.layers.wrappers import  Bidirectional
import h5py as h5
import matplotlib.pyplot as plt
from keras.models import Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import  Dense, Flatten, Activation, Dropout, Embedding, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Concatenate, BatchNormalization, GaussianNoise
from tensorflow.keras.layers import LSTM, TimeDistributed, Permute, Reshape, Lambda, RepeatVector, Input, Multiply, SimpleRNN, GRU, LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.utils import to_categorical
from tensorflow.summary import create_file_writer
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping

In [4]:
INPUT_DIM = 8    
TIME_STEPS = 1000  # The step of RNN
hf_Train = h5.File('./drive/MyDrive/data_papers/ncRNA/e2e_Train_Data_1000.h5', 'r')
hf_Val = h5.File('./drive/MyDrive/data_papers/ncRNA/e2e_Val_Data_1000.h5', 'r')
hf_Test = h5.File('./drive/MyDrive/data_papers/ncRNA/e2e_Test_Data_1000.h5', 'r')

X_train = hf_Train['Train_Data'] # Get train set
X_train = np.array(X_train)
Y_train = hf_Train['Label']      # Get train label
Y_train = np.array(Y_train)

X_val = hf_Val['Train_Data']     # Get validatoin set
X_val = np.array(X_val)
Y_val = hf_Val['Label']       # Get validation label
Y_val = np.array(Y_val)

X_test = hf_Test['Train_Data']     # Get test set
X_test = np.array(X_test)
Y_test = hf_Test['Label']       # Get test label
Y_test = np.array(Y_test)

Y_train = np_utils.to_categorical(Y_train, 13)  # Process the label of train
Y_val = np_utils.to_categorical(Y_val, 13)  # Process the label of train
Y_test = np_utils.to_categorical(Y_test, 13)    #  Process the label of test

In [5]:
Y_val.shape

(857, 13)

In [7]:
def attention_3d_block(inputs): # Attention Mechanism
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax', kernel_initializer='RandomNormal', bias_initializer='zeros')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul
    
def model_attention_applied_after_lstm(): # Model
    inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    lstm_units = 128
    lstm_out = Bidirectional(LSTM(lstm_units, return_sequences=True, kernel_initializer='RandomNormal', dropout= 0.3, recurrent_dropout = 0.3,recurrent_initializer='RandomNormal', bias_initializer='zero'))(inputs)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    dense_one = Dense(128, kernel_initializer='RandomNormal', bias_initializer='zeros', activation='relu')(attention_mul)
    dense_one = Dropout(0.4)(dense_one)
    dense_two = Dense(64, kernel_initializer='RandomNormal', bias_initializer='zeros', activation='relu')(dense_one)
    dense_two = Dropout(0.4)(dense_two)
    output = Dense(13, activation='softmax')(dense_two)
    model = Model(inputs=[inputs], outputs=output)
    return model


In [ ]:
import datetime

m = model_attention_applied_after_lstm()
m._name = f"ncRFP_{datetime.datetime.now():%Y%m%d%H%M%S}"

callbacks_used = []
callbacks_used.append(ModelCheckpoint(f'/content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/{m.name}' + '_model_{epoch:03d}_{accuracy:0.3f}',
                                      save_weights_only=False,
                                      monitor = 'accuracy', 
                                      mode='max',
                                      save_best_only=True))
callbacks_used.append(EarlyStopping(monitor='accuracy', patience=60))
callbacks_used.append(CSVLogger(f"/content/drive/MyDrive/ncRNA/data/history/history_log_{m.name}_{datetime.date.today().strftime('%Y%m%d')}.csv", append=True))

m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = m.fit(X_train, Y_train, callbacks=callbacks_used, verbose=2, batch_size=128, epochs=500, validation_data=(X_val, Y_val))

m.save(f"/content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/{m.name}_saved_model_after_fit")  # Save the model


# m.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
# history = m.fit(X_train, Y_train, batch_size=128, epochs=500, validation_data=(X_val, Y_val)) # Train 50 epochs
# m.save("Tenth_Fold_New_Model_500_8") #Save the model

Epoch 1/500
54/54 - 263s - loss: 2.4003 - accuracy: 0.1591 - val_loss: 1.9400 - val_accuracy: 0.3769
INFO:tensorflow:Assets written to: /content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/ncRFP_20210905210413_model_001_0.159/assets


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/500
54/54 - 258s - loss: 2.0340 - accuracy: 0.3139 - val_loss: 1.9249 - val_accuracy: 0.3349
INFO:tensorflow:Assets written to: /content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/ncRFP_20210905210413_model_002_0.314/assets


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 3/500
54/54 - 257s - loss: 1.8983 - accuracy: 0.3565 - val_loss: 1.7651 - val_accuracy: 0.3839
INFO:tensorflow:Assets written to: /content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/ncRFP_20210905210413_model_003_0.357/assets


...

Epoch 196/500
54/54 - 257s - loss: 0.0149 - accuracy: 0.9943 - val_loss: 0.9096 - val_accuracy: 0.8856
Epoch 197/500
54/54 - 258s - loss: 0.0118 - accuracy: 0.9961 - val_loss: 0.8745 - val_accuracy: 0.8845
Epoch 198/500
54/54 - 257s - loss: 0.0146 - accuracy: 0.9953 - val_loss: 0.9309 - val_accuracy: 0.8763
Epoch 199/500
54/54 - 258s - loss: 0.0109 - accuracy: 0.9969 - val_loss: 0.9735 - val_accuracy: 0.8670
Epoch 200/500
54/54 - 257s - loss: 0.0096 - accuracy: 0.9969 - val_loss: 0.9669 - val_accuracy: 0.8798
Epoch 201/500
54/54 - 257s - loss: 0.0121 - accuracy: 0.9955 - val_loss: 0.9252 - val_accuracy: 0.8798
Epoch 202/500
54/54 - 257s - loss: 0.0106 - accuracy: 0.9968 - val_loss: 0.9738 - val_accuracy: 0.8786
Epoch 203/500
54/54 - 256s - loss: 0.0093 - accuracy: 0.9968 - val_loss: 0.9516 - val_accuracy: 0.8798
Epoch 204/500
54/54 - 257s - loss: 0.0114 - accuracy: 0.9969 - val_loss: 1.0571 - val_accuracy: 0.8821
Epoch 205/500
54/54 - 256s - loss: 0.0201 - accuracy: 0.9943 - val_loss: 

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 233/500
54/54 - 258s - loss: 0.0091 - accuracy: 0.9974 - val_loss: 1.0103 - val_accuracy: 0.8716
Epoch 234/500
54/54 - 257s - loss: 0.0089 - accuracy: 0.9978 - val_loss: 1.0995 - val_accuracy: 0.8518
Epoch 235/500
54/54 - 257s - loss: 0.0120 - accuracy: 0.9959 - val_loss: 1.0507 - val_accuracy: 0.8623
Epoch 236/500
54/54 - 257s - loss: 0.0107 - accuracy: 0.9969 - val_loss: 1.0503 - val_accuracy: 0.8693
Epoch 237/500
54/54 - 257s - loss: 0.0097 - accuracy: 0.9966 - val_loss: 1.1346 - val_accuracy: 0.8716
Epoch 238/500
54/54 - 257s - loss: 0.0107 - accuracy: 0.9977 - val_loss: 0.9832 - val_accuracy: 0.8728
Epoch 239/500
54/54 - 258s - loss: 0.0114 - accuracy: 0.9971 - val_loss: 0.9452 - val_accuracy: 0.8751
Epoch 240/500
54/54 - 258s - loss: 0.0109 - accuracy: 0.9959 - val_loss: 0.9120 - val_accuracy: 0.8763
Epoch 241/500
54/54 - 261s - loss: 0.0166 - accuracy: 0.9958 - val_loss: 1.0585 - val_accuracy: 0.8681
Epoch 242/500
54/54 - 261s - loss: 0.0127 - accuracy: 0.9965 - val_loss: 

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 251/500
54/54 - 258s - loss: 0.0091 - accuracy: 0.9975 - val_loss: 1.0155 - val_accuracy: 0.8693
Epoch 252/500
54/54 - 257s - loss: 0.0073 - accuracy: 0.9975 - val_loss: 0.9765 - val_accuracy: 0.8728
Epoch 253/500
54/54 - 257s - loss: 0.0080 - accuracy: 0.9974 - val_loss: 0.9243 - val_accuracy: 0.8798
Epoch 254/500
54/54 - 257s - loss: 0.0048 - accuracy: 0.9984 - val_loss: 0.9463 - val_accuracy: 0.8786
Epoch 255/500
54/54 - 257s - loss: 0.0093 - accuracy: 0.9974 - val_loss: 1.0059 - val_accuracy: 0.8798
Epoch 256/500
54/54 - 258s - loss: 0.0103 - accuracy: 0.9975 - val_loss: 1.0334 - val_accuracy: 0.8833
Epoch 257/500
54/54 - 258s - loss: 0.0064 - accuracy: 0.9975 - val_loss: 1.0224 - val_accuracy: 0.8798
Epoch 258/500
54/54 - 257s - loss: 0.0098 - accuracy: 0.9966 - val_loss: 1.0986 - val_accuracy: 0.8728
Epoch 259/500
54/54 - 258s - loss: 0.0081 - accuracy: 0.9974 - val_loss: 0.9962 - val_accuracy: 0.8810
Epoch 260/500
54/54 - 257s - loss: 0.0130 - accuracy: 0.9975 - val_loss: 

In [1]:
history_dict = history.history
# Save it under the form of a json file
json.dump(history_dict, open(f'/content/drive/MyDrive/ncRNA/data/history/history_dict_{m.name}.json', 'w'))

NameError: ignored

In [ ]:
# /content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/ncRFP_20210905210413_model_250_0.999